# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
import json
from pprint import pprint

# Import API key
sys.path.append("..")
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create a path
csv_data_file = os.path.join("..","WeatherPy", "output_data", "cities.csv")

#Open csv file
with open(csv_data_file, "r", encoding= "utf-8") as read_file:
    weather_data = pd.read_csv(read_file, na_filter = False)
        
read_file.close()

# Dilsplay data
weather_data

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_name
0,0,,33.08,-16.33,71.60,78,75,12.75,PT,1593628934,Camacha
1,2,,-42.88,147.33,53.60,66,75,13.87,AU,1593628840,Hobart
2,3,,39.57,3.21,84.20,74,0,10.29,ES,1593628934,Manacor
3,4,,-8.90,-35.15,78.53,68,0,7.29,BR,1593628935,São José da Coroa Grande
4,6,,33.59,-102.38,97.00,16,75,9.35,US,1593628935,Levelland
...,...,...,...,...,...,...,...,...,...,...,...
369,544,,-1.80,37.62,64.24,72,100,6.26,KE,1593629193,Makueni Boma
370,545,,21.35,-158.09,82.40,54,1,6.93,US,1593629194,Makakilo City
371,546,,13.47,22.20,86.67,41,100,9.80,TD,1593629194,Adré
372,547,,20.45,-101.52,71.60,60,75,9.17,MX,1593629194,Abasolo


In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key = g_key)

# Convert Humidity to float and store
weather_data = weather_data.dropna()
humidity = weather_data["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into locations
locations = weather_data[["Latitude", "Longitude"]]

#cate a humidity Heatmap Layer
fig = gmaps.figure(zoom_level=2, center=[0,0], map_type='TERRAIN', layout={'width': '600px', 'height': '500px'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,
                               max_intensity=max(weather_data["Humidity"]), point_radius =2)
# Add heatmap layer to the figure
fig.add_layer(heat_layer)

# Display gmaps figure
fig

Figure(layout=FigureLayout(height='500px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Create a DataFrame to find your ideal weather condition.
vacation_data = weather_data
vacation_data = weather_data.loc[(weather_data["Max Temp"] >=60)&(weather_data["Max Temp"] <65)&
                                 (weather_data["Cloudiness"] < 5)&(weather_data["Wind Speed"] < 10)]
print(len(vacation_data))

vacation_data.head()




10


,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_name
10,12,,-33.59,26.89,64.00,54,0,3.00,ZA,1593628937,Port Alfred
11,13,,-17.88,15.07,64.38,28,0,6.38,NA,1593628937,Ongandjera
30,45,,59.64,-151.55,60.80,63,1,2.95,US,1593628949,Homer
58,89,,-26.79,-60.44,62.02,43,0,4.03,AR,1593628958,Presidencia Roque Sáenz Peña
141,205,,57.79,-152.41,60.80,67,1,1.34,US,1593629016,Saint Paul Harbor


###### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Copy DataFrame
hotel_df = vacation_data

hotel_df['Hotel Name'] = ''

# URL for API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

# Params dictionary 
params = {'type': 'hotel', 
    'key': g_key, 
    'radius': 5000}


# List for the hotel names

hotel_names = []

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    locate = f'{row["Latitude"]},{row["Longitude"]}'
    
    params['location'] = locate
     
    # assemble url and make API request
    print(f'Retrieving Results for Index {index}: {row["City_name"]}.')
    
    response_json = requests.get(base_url, params=params).json()

    # extract results
    results = response_json['results']

# Loop through results to get the first hotel
    for i in range(0, len(results)): 
        try:
        
            if results[i]['status'] == 'AVAIL':
                print(f'Closest hotel is {results[i]["name"]}.')
        
                hotel_names.append(results[i]["name"])
                break
     
        except:
            i += 
        
    print('-----------------------------')
    
hotel_df['Hotel Name'] = hotel_names


In [ ]:
# Display DataFrame
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)



# Display figure
fig